In [1]:
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
import sys
sys.path.append("../../../")

%run ../../../ai/nas/everyone/micro/model_inputs.py
%run ../../../ai/tf/setup/sess.py

In [3]:
def load_dataset(tf_src_path, batch_size=8):
    tfrecord_files = glob.glob(os.path.join(tf_src_path, '*.tfr'))
    data_iter = to_dataset(tfrecord_files, batch_size=batch_size)
    uid, frame, face, le, re, so, cam_x, cam_y, cam_to_x, cam_to_y, candide = data_iter.get_next()

    tensor_bag = {
        "uid": uid, "frame": frame, "face": face, "le": le, "re": re, "so": so,
        "cam_x": cam_x, "cam_y": cam_y, "cam_to_x": cam_to_x, "cam_to_y": cam_to_y, "candide": candide
    }

    return data_iter, tensor_bag

def feature_selector(tensors):
    return [
        tensors['le'],
        tensors['re'],
        tensors['candide'],
        tensors['cam_x'],
        tensors['cam_y']
    ]

# load dataset

In [4]:
batch_size = 8
tf_path = "/home/chy/archive-data/processed/everyone-tfr-candide"
data_iter, tensor_bag = load_dataset(tf_path, batch_size=batch_size)
inputs = feature_selector(tensor_bag)

Instructions for updating:
Use the `axis` argument instead


In [29]:
size = 1
x_train = [[]] * 3
y_train = []
x_test = [[]] * 3
y_test = []

with tf.Session(config=get_sess_config()) as sess:
    sess.run(data_iter.initializer)

    for i in range(size):
        le, re, candide, cam_x, cam_y = sess.run(inputs)
        for j in range(batch_size):
            x_train[0].append(le[j])
            x_train[1].append(re[j])
            x_train[2].append(candide[j])
            y_train.append((cam_x[j], cam_y[j]))

    for i in range(size):
        le, re, candide, cam_x, cam_y = sess.run(inputs)
        for j in range(batch_size):
            x_test[0].append(le[j])
            x_test[1].append(re[j])
            x_test[2].append(candide[j])
            y_test.append((cam_x[j], cam_y[j]))

print("*** prepare dataset complete ***")

*** prepare dataset complete ***


In [31]:
df_train_x = pd.DataFrame(x_train)
df_train_y = pd.DataFrame(y_train)
df_test_x = pd.DataFrame(x_test)
df_test_y = pd.DataFrame(y_test)

tx = df_train_x.values
ty = df_train_y.values
vx = df_test_x.values
vy = df_test_y.values

# build ref-model

In [32]:
df_train_x

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,"[[[119.0], [119.0], [118.625], [117.6875], [11...","[[[56.0], [60.8125], [60.75], [56.0], [45.0], ...","[203.67451, -0.65698546, 0.09975263, 0.0401114...","[[[145.0], [145.0], [145.0], [145.0], [145.0],...","[[[28.0], [32.178574], [37.142857], [42.714287...","[177.3606, -0.7454374, 0.07449965, -0.1901389,...","[[[124.0], [124.94643], [125.89286], [126.4196...","[[[181.0], [182.89285], [184.78572], [185.0], ...","[126.240616, -0.62487185, -0.040893428, 0.0863...","[[[122.0], [123.30357], [123.08929], [121.1339...",...,"[223.21838, -0.16121262, 0.12034277, -0.052775...","[[[109.0], [107.42857], [105.28571], [103.6428...","[[[145.0], [140.28572], [135.0], [130.92857], ...","[219.18542, 0.048399158, -0.57438916, 0.000533...","[[[177.0], [178.91072], [181.73215], [185.4642...","[[[59.0], [59.0], [59.910713], [61.732143], [6...","[267.62198, -0.07476014, 0.0088669835, -0.0299...","[[[139.0], [139.0], [139.0], [139.0], [139.178...","[[[125.0], [127.38393], [126.82143], [122.9285...","[48.754475, 0.11679276, -0.34843415, -0.083177..."
1,"[[[119.0], [119.0], [118.625], [117.6875], [11...","[[[56.0], [60.8125], [60.75], [56.0], [45.0], ...","[203.67451, -0.65698546, 0.09975263, 0.0401114...","[[[145.0], [145.0], [145.0], [145.0], [145.0],...","[[[28.0], [32.178574], [37.142857], [42.714287...","[177.3606, -0.7454374, 0.07449965, -0.1901389,...","[[[124.0], [124.94643], [125.89286], [126.4196...","[[[181.0], [182.89285], [184.78572], [185.0], ...","[126.240616, -0.62487185, -0.040893428, 0.0863...","[[[122.0], [123.30357], [123.08929], [121.1339...",...,"[223.21838, -0.16121262, 0.12034277, -0.052775...","[[[109.0], [107.42857], [105.28571], [103.6428...","[[[145.0], [140.28572], [135.0], [130.92857], ...","[219.18542, 0.048399158, -0.57438916, 0.000533...","[[[177.0], [178.91072], [181.73215], [185.4642...","[[[59.0], [59.0], [59.910713], [61.732143], [6...","[267.62198, -0.07476014, 0.0088669835, -0.0299...","[[[139.0], [139.0], [139.0], [139.0], [139.178...","[[[125.0], [127.38393], [126.82143], [122.9285...","[48.754475, 0.11679276, -0.34843415, -0.083177..."
2,"[[[119.0], [119.0], [118.625], [117.6875], [11...","[[[56.0], [60.8125], [60.75], [56.0], [45.0], ...","[203.67451, -0.65698546, 0.09975263, 0.0401114...","[[[145.0], [145.0], [145.0], [145.0], [145.0],...","[[[28.0], [32.178574], [37.142857], [42.714287...","[177.3606, -0.7454374, 0.07449965, -0.1901389,...","[[[124.0], [124.94643], [125.89286], [126.4196...","[[[181.0], [182.89285], [184.78572], [185.0], ...","[126.240616, -0.62487185, -0.040893428, 0.0863...","[[[122.0], [123.30357], [123.08929], [121.1339...",...,"[223.21838, -0.16121262, 0.12034277, -0.052775...","[[[109.0], [107.42857], [105.28571], [103.6428...","[[[145.0], [140.28572], [135.0], [130.92857], ...","[219.18542, 0.048399158, -0.57438916, 0.000533...","[[[177.0], [178.91072], [181.73215], [185.4642...","[[[59.0], [59.0], [59.910713], [61.732143], [6...","[267.62198, -0.07476014, 0.0088669835, -0.0299...","[[[139.0], [139.0], [139.0], [139.0], [139.178...","[[[125.0], [127.38393], [126.82143], [122.9285...","[48.754475, 0.11679276, -0.34843415, -0.083177..."


#### vemi checkpoint로 진행

In [22]:
model = build_model()
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[mean_pred, 'accuracy'])

history = TimeHistory()

# train